In [ ]:
import { assistantId } from "@/app/assistant-config";
import { openai } from "@/app/openai";
import fs from "fs";


const FILE_PATH = "app/api/assistants/codeInterpreter/asst_SnRuGNxq7U7uryBtKEuqbvkl.json";
if (!fs.existsSync(FILE_PATH)) {
  fs.writeFileSync(FILE_PATH, JSON.stringify([]), 'utf8'); // Creates an empty JSON file
};
  
// Type definition for stored file metadata
type FileEntry = { fileId: string; filename: string; assistantId: string };

const readData = (): FileEntry[] => {
    const data = fs.readFileSync(FILE_PATH, "utf8");
    return JSON.parse(data);
};
const writeData = (data: FileEntry[]) => {
    fs.writeFileSync(FILE_PATH, JSON.stringify(data, null, 2), "utf8");
};

// upload file to assistant's code interpreter
export async function POST(request: { formData: () => any; }) {
  const formData = await request.formData(); // process file as FormData
  const file = formData.get("file") as File; // retrieve the single file from FormData

// upload using the file stream
const openaiFile = await openai.files.create({
  file: file,
  purpose: "assistants",
});
// Update an assistant using the file ID

// Store file metadata
const files = readData();
files.push({ fileId: openaiFile.id, filename: file.name, assistantId });
writeData(files);
return new Response(JSON.stringify({ success: true, fileId: openaiFile.id, filename: file.name, assistantId }), {
  status: 200,
  headers: { "Content-Type": "application/json" },
});
}

// list files in assistant's file storage
export async function GET() {

    const fileList = readData() || []; // Ensure it's always an array
    const filteredFiles = fileList.filter(
      (file) => file?.filename && (file.filename.endsWith(".csv") || file.filename.endsWith(".xlsx"))
    );

    const filesArray = await Promise.all(
      filteredFiles.map(async (file) => {
        const fileDetails = await openai.files.retrieve(file.fileId);
        return {
          file_id: file.fileId,
          filename: fileDetails.filename,
        };
      })
    );

    const fileIds: string[] = filteredFiles.map((file) => file.fileId);

    await openai.beta.assistants.update(assistantId, {
      tool_resources: {
        code_interpreter: {
          file_ids: fileIds,
        },
      },
    });

    return new Response(JSON.stringify(filesArray), {
      headers: { "Content-Type": "application/json" },
    });
  
}

// delete file from assistant's vector store
export async function DELETE(request) {
  const body = await request.json();
  const fileId = body.fileId;
   // Delete file from OpenAI
   await openai.files.del(fileId); // delete file
  let files = readData();
  files = files.filter(file => file.fileId !== fileId);
  writeData(files);
  
  return new Response(JSON.stringify({ success: true, fileId }), {
    status: 200,
    headers: { "Content-Type": "application/json" },
  });
}

TypeError: Relative import path "@/app/assistant-config" not prefixed with / or ./ or ../

In [7]:
import fs from "fs";
import path from "path";


TypeError: Relative import path "fs" not prefixed with / or ./ or ../

In [9]:
import fs from "fs";
import path from "path";

// Define the path to your JSON file
const FILE_PATH = path.join(process.cwd(), "app/api/assistants/codeInterpreter/asst_SnRuGNxq7U7uryBtKEuqbvkl.json");

// Check if the file exists, if not, create it with an empty array
if (!fs.existsSync(FILE_PATH)) {
  fs.writeFileSync(FILE_PATH, JSON.stringify([]), 'utf8');
}

// Type definition for stored file metadata
type FileEntry = { fileId: string; filename: string; assistantId: string };

// Function to read data from the JSON file
const readData = (): FileEntry[] => {
  try {
    const data = fs.readFileSync(FILE_PATH, "utf8");
    return JSON.parse(data); // Return the parsed data
  } catch (error) {
    console.error("Error reading data:", error);
    return []; // Return an empty array if reading fails
  }
};

// Function to write data to the JSON file
const writeData = (data: FileEntry[]) => {
  try {
    fs.writeFileSync(FILE_PATH, JSON.stringify(data, null, 2), "utf8");
  } catch (error) {
    console.error("Error writing data:", error);
  }
};

// Example to track last deleted file ID (this should probably be part of some logic)
let lastDeletedFileId: string | null = null;
lastDeletedFileId = "abcd3432"; // Store last deleted file ID

console.log(lastDeletedFileId); // Just logging for demonstration

// Example API route handling
export async function POST(req: Request) {
  try {
    const body = await req.json(); // Get the JSON body from the request
    const { fileId } = body; // Extract fileId from request body

    // Example: Reading data, processing, and updating the file
    let data = readData();
    // You can now perform operations on the data, e.g., remove a file entry
    data = data.filter((entry) => entry.fileId !== fileId); // Removing file by fileId

    writeData(data); // Write the updated data back to the file

    return new Response(JSON.stringify({ success: true }), { status: 200 });
  } catch (error) {
    console.error("Error in POST handler:", error);
    return new Response(JSON.stringify({ error: "Internal server error" }), { status: 500 });
  }
}


TypeError: Relative import path "fs" not prefixed with / or ./ or ../